In [7]:
# Library imports
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing import image

In [8]:
# Dataset directory configuration
TRAIN_DIR = "./archive/train"
TEST_DIR = "./archive/test"
PREDICT_DIR = "./archive/predict"

# Verify directories exist
if not os.path.exists(TRAIN_DIR):
    raise ValueError(f"[ERROR] Training directory not found: {TRAIN_DIR}")
if not os.path.exists(TEST_DIR):
    raise ValueError(f"[ERROR] Test directory not found: {TEST_DIR}")

print(f"[INFO] Dataset directories configured successfully")
print(f"       >> Train directory: {TRAIN_DIR}")
print(f"       >> Test directory: {TEST_DIR}")

[INFO] Dataset directories configured successfully
       >> Train directory: ./archive/train
       >> Test directory: ./archive/test


In [9]:
# Image parameters and hyperparameters
IMG_HEIGHT = 224
IMG_WIDTH = 224
IMG_SIZE = (IMG_HEIGHT, IMG_WIDTH)
BATCH_SIZE = 32
EPOCHS = 15
LEARNING_RATE = 0.0001
VALIDATION_SPLIT = 0.2

print(f"[INFO] Hyperparameters configured:")
print(f"       >> Image size: {IMG_SIZE}")
print(f"       >> Batch size: {BATCH_SIZE}")
print(f"       >> Epochs: {EPOCHS}")
print(f"       >> Learning rate: {LEARNING_RATE}")
print(f"       >> Validation split: {VALIDATION_SPLIT}")

[INFO] Hyperparameters configured:
       >> Image size: (224, 224)
       >> Batch size: 32
       >> Epochs: 15
       >> Learning rate: 0.0001
       >> Validation split: 0.2


In [10]:
# Data preprocessing and augmentation
# Training data with augmentation to prevent overfitting
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=VALIDATION_SPLIT
)

# Test data with only rescaling (no augmentation)
test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

# Create data generators
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training',
    shuffle=True
)

validation_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation',
    shuffle=False
)

test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

print(f"[INFO] Data generators created successfully")
print(f"       >> Training samples: {train_generator.samples}")
print(f"       >> Validation samples: {validation_generator.samples}")
print(f"       >> Test samples: {test_generator.samples}")
print(f"       >> Class indices: {train_generator.class_indices}")

Found 3788 images belonging to 2 classes.
Found 945 images belonging to 2 classes.
Found 945 images belonging to 2 classes.
Found 1184 images belonging to 2 classes.
[INFO] Data generators created successfully
       >> Training samples: 3788
       >> Validation samples: 945
       >> Test samples: 1184
       >> Class indices: {'chihuahua': 0, 'muffin': 1}
Found 1184 images belonging to 2 classes.
[INFO] Data generators created successfully
       >> Training samples: 3788
       >> Validation samples: 945
       >> Test samples: 1184
       >> Class indices: {'chihuahua': 0, 'muffin': 1}


In [11]:
# ResNet CNN model architecture
def build_resnet_model(img_height, img_width, learning_rate):
    """
    Build an optimized ResNet-based CNN model for binary classification.
    
    Uses transfer learning with ResNet50 pre-trained on ImageNet.
    The base model is fine-tuned with custom top layers.
    
    Args:
        img_height (int): Height of input images
        img_width (int): Width of input images
        learning_rate (float): Learning rate for optimizer
    
    Returns:
        Model: Compiled Keras model
    """
    # Load pre-trained ResNet50 without top layers
    base_model = ResNet50(
        weights='imagenet',
        include_top=False,
        input_shape=(img_height, img_width, 3)
    )
    
    # Freeze the base model initially
    base_model.trainable = False
    
    # Build custom top layers
    inputs = layers.Input(shape=(img_height, img_width, 3))
    x = base_model(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    
    # Create the complete model
    model = Model(inputs=inputs, outputs=outputs)
    
    # Compile model with optimizer and loss function
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
    )
    
    print(f"[INFO] ResNet model built successfully")
    print(f"       >> Total layers: {len(model.layers)}")
    print(f"       >> Base model trainable: {base_model.trainable}")
    
    return model, base_model

# Build the model
model, base_model = build_resnet_model(IMG_HEIGHT, IMG_WIDTH, LEARNING_RATE)

[INFO] ResNet model built successfully
       >> Total layers: 9
       >> Base model trainable: False


In [12]:
# Display model architecture summary
print("[INFO] Model Architecture Summary:")
model.summary()

[INFO] Model Architecture Summary:


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 2048)           │         8,192 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,153,473 (92.14 MB)

 Trainable params: 561,665 (2.14 MB)

 Non-trainable params: 23,591,808 (90.00 MB)

In [13]:
# Configure training callbacks for optimization
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-7,
    verbose=1
)

callbacks_list = [early_stopping, reduce_lr]

print(f"[INFO] Training callbacks configured")
print(f"       >> Early stopping: patience={early_stopping.patience}")
print(f"       >> Learning rate reduction: factor={reduce_lr.factor}, patience={reduce_lr.patience}")

[INFO] Training callbacks configured
       >> Early stopping: patience=5
       >> Learning rate reduction: factor=0.5, patience=3


In [14]:
# Train the model - Phase 1: Train with frozen base model
print("[INFO] Starting training - Phase 1: Frozen base model")
print("=" * 60)

history_phase1 = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator,
    callbacks=callbacks_list,
    verbose=1
)

print("\n[INFO] Phase 1 training completed")

[INFO] Starting training - Phase 1: Frozen base model


c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 469s 4s/step - accuracy: 0.6988 - loss: 0.5942 - precision_1: 0.6720 - recall_1: 0.6724 - val_accuracy: 0.4963 - val_loss: 0.7351 - val_precision_1: 0.4769 - val_recall_1: 1.0000 - learning_rate: 1.0000e-04
Epoch 2/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 469s 4s/step - accuracy: 0.6988 - loss: 0.5942 - precision_1: 0.6720 - recall_1: 0.6724 - val_accuracy: 0.4963 - val_loss: 0.7351 - val_precision_1: 0.4769 - val_recall_1: 1.0000 - learning_rate: 1.0000e-04
Epoch 2/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 313s 3s/step - accuracy: 0.7416 - loss: 0.5377 - precision_1: 0.7272 - recall_1: 0.7000 - val_accuracy: 0.7249 - val_loss: 0.5750 - val_precision_1: 0.6355 - val_recall_1: 0.9401 - learning_rate: 1.0000e-04
Epoch 3/15
119/119 ━━━━━━━━━━━━━━━━━━━━ 313s 3s/step - accuracy: 0.7416 - loss: 0.5377 - precision_1: 0.7272 - recall_1: 0.7000 - val_accuracy: 0.7249 - val_loss: 0.5750 - val_precision_1: 0.6355 - val_recall_1: 0.9401 - learning_rate: 1.0000e-04
Epoch 3/15
1

In [17]:
# Fine-tuning: Unfreeze the base model for better performance
print("[INFO] Starting fine-tuning - Phase 2: Unfreezing base model")
print("=" * 60)

# Unfreeze the base model
base_model.trainable = True

# Freeze the first 100 layers to retain low-level features
for layer in base_model.layers[:100]:
    layer.trainable = False

# Recompile with a lower learning rate for fine-tuning
fine_tune_lr = LEARNING_RATE / 10
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=fine_tune_lr),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

print(f"[INFO] Model recompiled for fine-tuning")
print(f"       >> Fine-tuning learning rate: {fine_tune_lr}")
print(f"       >> Trainable layers: {sum([1 for layer in model.layers if layer.trainable])}")

# Continue training with unfrozen layers
history_phase2 = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    callbacks=callbacks_list,
    verbose=1
)

print("\n[INFO] Phase 2 fine-tuning completed")

[INFO] Starting fine-tuning - Phase 2: Unfreezing base model
[INFO] Model recompiled for fine-tuning
       >> Fine-tuning learning rate: 1e-05
       >> Trainable layers: 9
Epoch 1/10
Epoch 1/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 1210s 10s/step - accuracy: 0.7405 - loss: 0.6123 - precision_4: 0.7276 - recall_4: 0.6954 - val_accuracy: 0.7259 - val_loss: 0.7251 - val_precision_4: 0.8995 - val_recall_4: 0.4539 - learning_rate: 1.0000e-05
Epoch 2/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 1210s 10s/step - accuracy: 0.7405 - loss: 0.6123 - precision_4: 0.7276 - recall_4: 0.6954 - val_accuracy: 0.7259 - val_loss: 0.7251 - val_precision_4: 0.8995 - val_recall_4: 0.4539 - learning_rate: 1.0000e-05
Epoch 2/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 798s 7s/step - accuracy: 0.7497 - loss: 0.6154 - precision_4: 0.7294 - recall_4: 0.7236 - val_accuracy: 0.7989 - val_loss: 0.4932 - val_precision_4: 0.7319 - val_recall_4: 0.8871 - learning_rate: 1.0000e-05
Epoch 3/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 798s 7s/step - accuracy: 0.749

In [18]:
# Evaluate the trained model on test data
print("[INFO] Evaluating model on test dataset")
print("=" * 60)

test_results = model.evaluate(test_generator, verbose=1)
test_loss = test_results[0]
test_accuracy = test_results[1]
test_precision = test_results[2]
test_recall = test_results[3]

print("\n[INFO] Test Results:")
print(f"       >> Test Loss: {test_loss:.4f}")
print(f"       >> Test Accuracy: {test_accuracy:.4f}")
print(f"       >> Test Precision: {test_precision:.4f}")
print(f"       >> Test Recall: {test_recall:.4f}")
print(f"       >> F1-Score: {2 * (test_precision * test_recall) / (test_precision + test_recall):.4f}")

[INFO] Evaluating model on test dataset
37/37 ━━━━━━━━━━━━━━━━━━━━ 73s 2s/step - accuracy: 0.7990 - loss: 0.5114 - precision_4: 0.9298 - recall_4: 0.6085

[INFO] Test Results:
       >> Test Loss: 0.5114
       >> Test Accuracy: 0.7990
       >> Test Precision: 0.9298
       >> Test Recall: 0.6085
       >> F1-Score: 0.7356
37/37 ━━━━━━━━━━━━━━━━━━━━ 73s 2s/step - accuracy: 0.7990 - loss: 0.5114 - precision_4: 0.9298 - recall_4: 0.6085

[INFO] Test Results:
       >> Test Loss: 0.5114
       >> Test Accuracy: 0.7990
       >> Test Precision: 0.9298
       >> Test Recall: 0.6085
       >> F1-Score: 0.7356


In [19]:
# Save the trained model
MODEL_SAVE_PATH = 'exercise_6_trained_resnet.h5'

model.save(MODEL_SAVE_PATH)
print(f"[INFO] Model saved successfully to: {MODEL_SAVE_PATH}")

[INFO] Model saved successfully to: exercise_6_trained_resnet.h5


In [20]:
# Image prediction function
def predict_image(img_path, model_path='exercise_6_trained_resnet.h5', img_size=(224, 224)):
    """
    Predict whether an image is a muffin or chihuahua using the trained ResNet model.
    
    Args:
        img_path (str): Path to the image file
        model_path (str): Path to the saved model file
        img_size (tuple): Size to resize the image to (height, width)
    
    Returns:
        None: Prints the prediction result
    """
    try:
        # Load the trained model
        loaded_model = tf.keras.models.load_model(model_path)
        
        # Load and preprocess the image
        img = image.load_img(img_path, target_size=img_size)
        img_array = image.img_to_array(img)
        img_array = img_array / 255.0
        img_array = np.expand_dims(img_array, axis=0)
        
        # Make prediction
        prediction = loaded_model.predict(img_array, verbose=0)[0, 0]
        
        # Interpret prediction (assumes class indices: {chihuahua: 0, muffin: 1})
        label = "Chihuahua" if prediction >= 0.5 else "Muffin"
        confidence = prediction if prediction >= 0.5 else (1 - prediction)
        
        print(f"[PREDICTION] {img_path}")
        print(f"             >> Result: {label}")
        print(f"             >> Confidence: {confidence:.2%}")
        print(f"             >> Raw score: {prediction:.4f}\n")
        
    except FileNotFoundError:
        print(f"[ERROR] Image file not found: {img_path}")
    except Exception as e:
        print(f"[ERROR] Prediction failed for {img_path}: {str(e)}")

print("[INFO] Prediction function defined successfully")

[INFO] Prediction function defined successfully


In [21]:
# Test predictions on sample images
print("[INFO] Running predictions on test images")
print("=" * 60)

predict_image("archive/predict/predict_1.png")
predict_image("archive/predict/predict_2.png")
predict_image("archive/predict/predict_3.png")
predict_image("archive/predict/predict_4.png")
predict_image("archive/predict/predict_5.png")
predict_image("archive/predict/predict_6.png")
predict_image("archive/predict/predict_7.png")
predict_image("archive/predict/predict_8.png")

print("[INFO] All predictions completed")

[INFO] Running predictions on test images


[PREDICTION] archive/predict/predict_1.png
             >> Result: Muffin
             >> Confidence: 95.77%
             >> Raw score: 0.0423



[PREDICTION] archive/predict/predict_2.png
             >> Result: Muffin
             >> Confidence: 87.59%
             >> Raw score: 0.1241



[PREDICTION] archive/predict/predict_3.png
             >> Result: Muffin
             >> Confidence: 96.44%
             >> Raw score: 0.0356



[PREDICTION] archive/predict/predict_4.png
             >> Result: Muffin
             >> Confidence: 99.82%
             >> Raw score: 0.0018



[PREDICTION] archive/predict/predict_5.png
             >> Result: Muffin
             >> Confidence: 99.52%
             >> Raw score: 0.0048



[PREDICTION] archive/predict/predict_6.png
             >> Result: Muffin
             >> Confidence: 99.77%
             >> Raw score: 0.0023



[PREDICTION] archive/predict/predict_7.png
             >> Result: Chihuahua
             >> Confidence: 55.55%
             >> Raw score: 0.5555



[PREDICTION] archive/predict/predict_8.png
             >> Result: Muffin
             >> Confidence: 97.99%
             >> Raw score: 0.0201

[INFO] All predictions completed
